In [85]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time
driver=webdriver.Chrome('c:/pydata/chromedriver.exe')
from sklearn.preprocessing import MinMaxScaler

C:\Users\Admin\AppData\Local\Temp\ipykernel_22424\2482100624.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome('c:/pydata/chromedriver.exe')


In [4]:
df = pd.read_csv('업종별_href_code_list.csv')
df

,ind,top_5_ind_list,top_5_href,top_5_code
0,건설업,"['현대건설', 'GS건설', '대우건설', '한전KPS', 'DL이앤씨']","['https://finance.daum.net/quotes/A000720', 'h...","['000720', '006360', '047040', '051600', '3755..."
1,의약품,"['삼성바이오로직스', '셀트리온', 'SK바이오사이언스', '유한양행', '한미약품']","['https://finance.daum.net/quotes/A207940', 'h...","['207940', '068270', '302440', '000100', '1289..."
2,철강금속,"['POSCO홀딩스', '고려아연', '현대제철', '영풍', '동국제강']","['https://finance.daum.net/quotes/A005490', 'h...","['005490', '010130', '004020', '000670', '0012..."
3,증권,"['미래에셋증권', 'NH투자증권', '삼성증권', '메리츠증권', '키움증권']","['https://finance.daum.net/quotes/A006800', 'h...","['006800', '005940', '016360', '008560', '0394..."
4,금융업,"['KB금융', '신한지주', 'SK', '삼성생명', 'LG']","['https://finance.daum.net/quotes/A105560', 'h...","['105560', '055550', '034730', '032830', '0035..."
5,기계,"['두산에너빌리티', '한온시스템', '두산밥캣', '씨에스윈드', 'LIG넥스원']","['https://finance.daum.net/quotes/A034020', 'h...","['034020', '018880', '241560', '112610', '0795..."
6,화학,"['LG화학', 'SK이노베이션', 'S-Oil', '한화솔루션', 'LG생활건강']","['https://finance.daum.net/quotes/A051910', 'h...","['051910', '096770', '010950', '009830', '0519..."
7,서비스업,"['NAVER', '카카오', '삼성에스디에스', '크래프톤', '엔씨소프트']","['https://finance.daum.net/quotes/A035420', 'h...","['035420', '035720', '018260', '259960', '0365..."
8,전기전자,"['삼성전자', 'LG에너지솔루션', 'SK하이닉스', '삼성SDI', '삼성전자우']","['https://finance.daum.net/quotes/A005930', 'h...","['005930', '373220', '000660', '006400', '0059..."
9,운수장비,"['현대차', '기아', '현대모비스', '현대중공업', '한국조선해양']","['https://finance.daum.net/quotes/A005380', 'h...","['005380', '000270', '012330', '329180', '0095..."


In [22]:
import re
for val in df.top_5_code[0][1:-1].replace("'", '').split(','):
    print(val.strip())

000720
006360
047040
051600
375500


In [72]:
# 주가 크롤링 함수
def stock_crawler(code_value, company_name):
    page=1

    # 삼성전자 005930
    code=f'{code_value}'
    url='https://finance.naver.com/item/sise_day.nhn?code='+ code + '&page='

    driver=webdriver.Chrome('c:/pydata/chromedriver.exe')
    driver.get(url+str(page))
    html = bs(driver.page_source, 'html.parser')
    endPage=html.find('td', class_="pgRR").find('a')['href'].split('=')[-1]

    nav_finance=[]

    for page in range(1, int(endPage)+1):
        driver.get(url+str(page))
        html = bs(driver.page_source, 'html.parser')
        soup_tr=html.find_all('tr')
        for i in range(2, 15):
            day_data=[]
            soup_td=soup_tr[i].find_all('td')
            if len(soup_td) < 7 :
                continue
            else:
                for td_txt in soup_td:
                    if td_txt.text == u'\xa0':
                        continue
                    else:
                        day_data.append(td_txt.text.replace('.', '-').replace('\n','').replace('\t', '').replace(',', ''))
                        # day_data.append(re('[가-힣0-9a-zA-Z]+',td_txt.text))

            nav_finance.append(day_data)

    driver.close()

    df=pd.DataFrame(nav_finance, columns=['날짜',company_name,"전일비",'시가','고가','저가','거래량'])
    # print(df.info())
    df=df.dropna()
    # df['날짜']=pd.to_datetime(df['날짜'])
    df['날짜'] = [ pd.to_datetime(df.iloc[date, 0]).strftime('%Y%m%d') for date in range(len(df))]
    df = df.sort_values(by='날짜')
    df = df[['날짜', company_name]]
    return df
# 파일 저장 코드
#today = datetime.today().strftime('%Y_%m_%d')
# df.to_csv(f'./삼성전자_{today}.csv', index=False)

In [80]:
# 코스피 크롤링
page=1
code='KOSPI'
url='https://finance.naver.com/sise/sise_index_day.naver?code='+ code + '&page='

driver=webdriver.Chrome('c:/pydata/chromedriver.exe')
driver.get(url+str(page))
html = bs(driver.page_source, 'html.parser')
endPage=html.find('td', class_="pgRR").find('a')['href'].split('=')[-1]

nav_finance=[]

for page in range(1, int(endPage)+1):
    driver.get(url+str(page))
    html = bs(driver.page_source, 'html.parser')
    soup_tr=html.find_all('tr')
    for i in range(2, 14):
        day_data=[]
        soup_td=soup_tr[i].find_all('td')
        if len(soup_td) < 6 :
            continue
        else:
            for td_txt in soup_td:
                if td_txt.text == u'\xa0':
                    continue
                else:
                    day_data.append(td_txt.text.replace('.', '-').replace('\n','').replace('\t', '').replace(',', ''))
                    # day_data.append(re('[가-힣0-9a-zA-Z]+',td_txt.text))

        nav_finance.append(day_data)

driver.close()
df_KOSPI=pd.DataFrame(nav_finance, columns=['날짜','KOSPI',"전일비",'등락률', '거래량','거래대금'])
df_KOSPI=df_KOSPI.dropna()
df_KOSPI['날짜'] = [ pd.to_datetime(df_KOSPI.iloc[date, 0]).strftime('%Y%m%d') for date in range(len(df_KOSPI))]
df_KOSPI = df_KOSPI.sort_values(by='날짜', ascending=False)
df_KOSPI = df_KOSPI[['날짜', 'KOSPI']]
df_KOSPI_price = []
for idx in df_KOSPI.index :
    df_KOSPI_price.append(df_KOSPI.loc[idx, 'KOSPI'].replace('-', ''))
df_KOSPI['KOSPI'] = df_KOSPI_price
df_KOSPI

C:\Users\Admin\AppData\Local\Temp\ipykernel_20980\983741572.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome('c:/pydata/chromedriver.exe')


,날짜,체결가
0,20221024,223616
1,20221021,221312
2,20221020,221809
3,20221019,223744
4,20221018,224995
...,...,...
8513,19900109,92021
8514,19900108,91946
8515,19900106,91286
8516,19900105,91511


In [81]:
# 복사본 생성
df_KOSPI_merge = df_KOSPI.copy()

In [82]:
# 모든 종목 크롤링 및 코스피랑 합치기
for idx in df.index :
    stock_code_list = df.loc[idx, 'top_5_code'][1:-1].replace("'", '').split(',')
    stock_name_list = df.loc[idx, 'top_5_ind_list'][1:-1].replace("'", '').split(',')
    for val_nm in range(len(stock_code_list)):
        df_temp = stock_crawler(stock_code_list[val_nm].strip(), stock_name_list[val_nm].strip())
        df_KOSPI_merge = pd.merge(df_KOSPI_merge, df_temp, how='left', on='날짜')
df_KOSPI_merge

C:\Users\Admin\AppData\Local\Temp\ipykernel_20980\4062798901.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome('c:/pydata/chromedriver.exe')
C:\Users\Admin\AppData\Local\Temp\ipykernel_20980\4062798901.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome('c:/pydata/chromedriver.exe')
C:\Users\Admin\AppData\Local\Temp\ipykernel_20980\4062798901.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome('c:/pydata/chromedriver.exe')
C:\Users\Admin\AppData\Local\Temp\ipykernel_20980\4062798901.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome('c:/pydata/chromedriver.exe')
C:\Users\Admin\AppData\Local\Temp\ipykernel_20980\4062798901.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service obj

,날짜,체결가,현대건설,GS건설,대우건설,한전KPS,DL이앤씨,삼성바이오로직스,셀트리온,SK바이오사이언스,...,삼성물산,BGF리테일,호텔신라,포스코인터내셔널,GS리테일,CJ제일제당,오리온,하이트진로,농심,오뚜기
0,20221024,223616,34950,22450,4165,32150,36450,873000,178000,75200,...,111000,184000,72000,21050,23950,398000,99400,24600,296000,439500
1,20221021,221312,34150,21700,3975,31700,34550,847000,174000,71800,...,109500,183000,72500,20950,23650,398500,99600,24450,294500,435000
2,20221020,221809,36450,22950,4230,32900,36100,835000,171500,72700,...,110500,176000,73500,21250,24050,403500,99800,24950,289000,439500
3,20221019,223744,35700,23100,4215,31300,36000,847000,171500,74900,...,108500,175000,73500,21400,24250,403000,98900,25050,292000,443500
4,20221018,224995,37300,23800,4290,31550,37650,845000,176500,77800,...,111000,176000,74500,21700,24550,402000,100500,25200,295000,446500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8513,19900109,92021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8514,19900108,91946,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8515,19900106,91286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8516,19900105,91511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# 결과 저장
df_KOSPI_merge.to_csv('주식_대장주_크롤링_결과.csv', index=False)

In [13]:
# 저장한 파일 불러와서 작업
df_stock_result = pd.read_csv('주식_대장주_크롤링_결과.csv')
df_stock_result

,날짜,KOSPI,현대건설,GS건설,대우건설,한전KPS,DL이앤씨,삼성바이오로직스,셀트리온,SK바이오사이언스,...,삼성물산,BGF리테일,호텔신라,포스코인터내셔널,GS리테일,CJ제일제당,오리온,하이트진로,농심,오뚜기
0,20221024,223616,34950.0,22450.0,4165.0,32150.0,36450.0,873000.0,178000.0,75200.0,...,111000.0,184000.0,72000.0,21050.0,23950.0,398000.0,99400.0,24600.0,296000.0,439500.0
1,20221021,221312,34150.0,21700.0,3975.0,31700.0,34550.0,847000.0,174000.0,71800.0,...,109500.0,183000.0,72500.0,20950.0,23650.0,398500.0,99600.0,24450.0,294500.0,435000.0
2,20221020,221809,36450.0,22950.0,4230.0,32900.0,36100.0,835000.0,171500.0,72700.0,...,110500.0,176000.0,73500.0,21250.0,24050.0,403500.0,99800.0,24950.0,289000.0,439500.0
3,20221019,223744,35700.0,23100.0,4215.0,31300.0,36000.0,847000.0,171500.0,74900.0,...,108500.0,175000.0,73500.0,21400.0,24250.0,403000.0,98900.0,25050.0,292000.0,443500.0
4,20221018,224995,37300.0,23800.0,4290.0,31550.0,37650.0,845000.0,176500.0,77800.0,...,111000.0,176000.0,74500.0,21700.0,24550.0,402000.0,100500.0,25200.0,295000.0,446500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8513,19900109,92021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8514,19900108,91946,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8515,19900106,91286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8516,19900105,91511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df_stock_1year = df_stock_result[df_stock_result['날짜'] > 20211024].sort_values(by='날짜', ascending=False)
df_stock_1year

,날짜,KOSPI,현대건설,GS건설,대우건설,한전KPS,DL이앤씨,삼성바이오로직스,셀트리온,SK바이오사이언스,...,삼성물산,BGF리테일,호텔신라,포스코인터내셔널,GS리테일,CJ제일제당,오리온,하이트진로,농심,오뚜기
0,20221024,223616,34950.0,22450.0,4165.0,32150.0,36450.0,873000.0,178000.0,75200.0,...,111000.0,184000.0,72000.0,21050.0,23950.0,398000.0,99400.0,24600.0,296000.0,439500.0
1,20221021,221312,34150.0,21700.0,3975.0,31700.0,34550.0,847000.0,174000.0,71800.0,...,109500.0,183000.0,72500.0,20950.0,23650.0,398500.0,99600.0,24450.0,294500.0,435000.0
2,20221020,221809,36450.0,22950.0,4230.0,32900.0,36100.0,835000.0,171500.0,72700.0,...,110500.0,176000.0,73500.0,21250.0,24050.0,403500.0,99800.0,24950.0,289000.0,439500.0
3,20221019,223744,35700.0,23100.0,4215.0,31300.0,36000.0,847000.0,171500.0,74900.0,...,108500.0,175000.0,73500.0,21400.0,24250.0,403000.0,98900.0,25050.0,292000.0,443500.0
4,20221018,224995,37300.0,23800.0,4290.0,31550.0,37650.0,845000.0,176500.0,77800.0,...,111000.0,176000.0,74500.0,21700.0,24550.0,402000.0,100500.0,25200.0,295000.0,446500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,20211029,297068,50300.0,41350.0,6250.0,41850.0,135000.0,871000.0,200500.0,230000.0,...,114000.0,161500.0,87400.0,21700.0,32200.0,380000.0,118000.0,34800.0,283500.0,481000.0
243,20211028,300955,50200.0,41250.0,6380.0,42250.0,132500.0,886000.0,215500.0,245500.0,...,117000.0,162500.0,87700.0,21500.0,32350.0,386500.0,119000.0,35300.0,284000.0,485000.0
244,20211027,302549,50800.0,41400.0,6630.0,43600.0,135000.0,898000.0,219000.0,229000.0,...,120500.0,168500.0,89100.0,21800.0,32950.0,393000.0,120500.0,35750.0,288500.0,488500.0
245,20211026,304908,51500.0,42350.0,6650.0,43450.0,136500.0,879000.0,222500.0,227000.0,...,121500.0,168000.0,91300.0,22300.0,33050.0,394500.0,123500.0,36900.0,290000.0,489000.0


In [37]:
# null 값 확인
df_stock_1year.isna().sum()[df_stock_1year.isna().sum() > 0]

LG에너지솔루션    67
dtype: int64

In [38]:
df

,ind,top_5_ind_list,top_5_href,top_5_code
0,건설업,"['현대건설', 'GS건설', '대우건설', '한전KPS', 'DL이앤씨']","['https://finance.daum.net/quotes/A000720', 'h...","['000720', '006360', '047040', '051600', '3755..."
1,의약품,"['삼성바이오로직스', '셀트리온', 'SK바이오사이언스', '유한양행', '한미약품']","['https://finance.daum.net/quotes/A207940', 'h...","['207940', '068270', '302440', '000100', '1289..."
2,철강금속,"['POSCO홀딩스', '고려아연', '현대제철', '영풍', '동국제강']","['https://finance.daum.net/quotes/A005490', 'h...","['005490', '010130', '004020', '000670', '0012..."
3,증권,"['미래에셋증권', 'NH투자증권', '삼성증권', '메리츠증권', '키움증권']","['https://finance.daum.net/quotes/A006800', 'h...","['006800', '005940', '016360', '008560', '0394..."
4,금융업,"['KB금융', '신한지주', 'SK', '삼성생명', 'LG']","['https://finance.daum.net/quotes/A105560', 'h...","['105560', '055550', '034730', '032830', '0035..."
5,기계,"['두산에너빌리티', '한온시스템', '두산밥캣', '씨에스윈드', 'LIG넥스원']","['https://finance.daum.net/quotes/A034020', 'h...","['034020', '018880', '241560', '112610', '0795..."
6,화학,"['LG화학', 'SK이노베이션', 'S-Oil', '한화솔루션', 'LG생활건강']","['https://finance.daum.net/quotes/A051910', 'h...","['051910', '096770', '010950', '009830', '0519..."
7,서비스업,"['NAVER', '카카오', '삼성에스디에스', '크래프톤', '엔씨소프트']","['https://finance.daum.net/quotes/A035420', 'h...","['035420', '035720', '018260', '259960', '0365..."
8,전기전자,"['삼성전자', 'LG에너지솔루션', 'SK하이닉스', '삼성SDI', '삼성전자우']","['https://finance.daum.net/quotes/A005930', 'h...","['005930', '373220', '000660', '006400', '0059..."
9,운수장비,"['현대차', '기아', '현대모비스', '현대중공업', '한국조선해양']","['https://finance.daum.net/quotes/A005380', 'h...","['005380', '000270', '012330', '329180', '0095..."


In [ ]:
df.loc[idx, 'top_5_code'][1:-1].replace("'", '').split(',')

In [76]:
# 산업구분 별로 합치기
df_ind_1year = df_stock_1year[['날짜', 'KOSPI']]
for idx in df.index :
    industry = df.loc[idx, 'ind']
    com_list = df.loc[idx, 'top_5_ind_list'][1:-1].replace("'", '').split(',')
    com_list = list(map(lambda x : x.strip(), com_list))
    if industry == '전기전자':
        com_list.remove('LG에너지솔루션')
    df_temp = df_stock_1year[['날짜'] + com_list].groupby('날짜').mean().mean(axis=1).reset_index().sort_values(by='날짜', ascending=False)
    del df_temp['날짜']
    df_temp = df_temp.rename(columns = { 0 : industry})
    df_ind_1year = pd.concat([df_ind_1year, df_temp], axis=1)
df_ind_1year
        

,날짜,KOSPI,건설업,의약품,철강금속,증권,금융업,기계,화학,서비스업,전기전자,운수장비,유통업,음식료품
0,20221024,223616,55802.0,328640.0,334990.0,36732.0,103800.0,39200.0,522270.0,359100.0,241750.0,154260.0,87350.0,267040.0
1,20221021,221312,56090.0,333400.0,335410.0,37143.0,104200.0,39220.0,513430.0,364400.0,246025.0,156260.0,87230.0,266780.0
2,20221020,221809,55486.0,336120.0,330510.0,37053.0,103030.0,39400.0,489230.0,366300.0,246550.0,156700.0,86570.0,265250.0
3,20221019,223744,54516.0,335260.0,318360.0,36364.0,100800.0,38810.0,483700.0,357900.0,247150.0,154700.0,84210.0,261960.0
4,20221018,224995,54950.0,325080.0,316100.0,35768.0,99270.0,38570.0,478570.0,356400.0,242700.0,153020.0,83360.0,259460.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,20211029,297068,26918.0,279380.0,306390.0,26381.0,86840.0,40290.0,295980.0,176880.0,202300.0,125940.0,81550.0,253840.0
243,20211028,300955,26063.0,277880.0,308880.0,25932.0,87010.0,40718.0,294280.0,176460.0,201600.0,125800.0,80530.0,252490.0
244,20211027,302549,26526.0,275120.0,305370.0,24284.0,87110.0,40798.0,284920.0,174150.0,196675.0,124860.0,81060.0,251350.0
245,20211026,304908,25215.0,278260.0,296840.0,23742.0,85670.0,40368.0,286630.0,174770.0,206350.0,123280.0,81920.0,250410.0


In [80]:
df_ind_1year[ind_list]

,건설업,의약품,철강금속,증권,금융업,기계,화학,서비스업,전기전자,운수장비,유통업,음식료품
0,55802.0,328640.0,334990.0,36732.0,103800.0,39200.0,522270.0,359100.0,241750.0,154260.0,87350.0,267040.0
1,56090.0,333400.0,335410.0,37143.0,104200.0,39220.0,513430.0,364400.0,246025.0,156260.0,87230.0,266780.0
2,55486.0,336120.0,330510.0,37053.0,103030.0,39400.0,489230.0,366300.0,246550.0,156700.0,86570.0,265250.0
3,54516.0,335260.0,318360.0,36364.0,100800.0,38810.0,483700.0,357900.0,247150.0,154700.0,84210.0,261960.0
4,54950.0,325080.0,316100.0,35768.0,99270.0,38570.0,478570.0,356400.0,242700.0,153020.0,83360.0,259460.0
...,...,...,...,...,...,...,...,...,...,...,...,...
242,26918.0,279380.0,306390.0,26381.0,86840.0,40290.0,295980.0,176880.0,202300.0,125940.0,81550.0,253840.0
243,26063.0,277880.0,308880.0,25932.0,87010.0,40718.0,294280.0,176460.0,201600.0,125800.0,80530.0,252490.0
244,26526.0,275120.0,305370.0,24284.0,87110.0,40798.0,284920.0,174150.0,196675.0,124860.0,81060.0,251350.0
245,25215.0,278260.0,296840.0,23742.0,85670.0,40368.0,286630.0,174770.0,206350.0,123280.0,81920.0,250410.0


In [87]:
df_ind_1year_sc = df_ind_1year.copy()
ind_list = df_ind_1year_sc.columns.drop(['날짜', 'KOSPI'])
for col in ind_list :
    df_ind_1year_sc[col] = df_ind_1year_sc[col] / df_ind_1year_sc.KOSPI
scaler = MinMaxScaler()
df_ind_1year_sc[ind_list] = scaler.fit_transform(df_ind_1year_sc[ind_list])
df_ind_1year_sc

,날짜,KOSPI,건설업,의약품,철강금속,증권,금융업,기계,화학,서비스업,전기전자,운수장비,유통업,음식료품
0,20221024,223616,0.956557,0.883469,0.972665,0.905936,0.966607,0.877671,1.000000,0.830920,0.850735,0.943116,0.974276,0.970698
1,20221021,221312,0.978912,0.937668,1.000000,0.943340,1.000000,0.911061,0.988830,0.862902,0.906413,0.995685,1.000000,1.000000
2,20221020,221809,0.960045,0.950755,0.960174,0.935142,0.968167,0.918286,0.907138,0.866737,0.906187,0.996986,0.971836,0.975015
3,20221019,223744,0.922787,0.925798,0.855185,0.887704,0.897840,0.845536,0.875841,0.825980,0.893563,0.948211,0.870216,0.909821
4,20221018,224995,0.926079,0.846907,0.827127,0.850453,0.851044,0.810128,0.850954,0.813742,0.846358,0.911549,0.827368,0.863955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,20211029,297068,0.045379,0.104226,0.243438,0.114697,0.102279,0.186027,0.042846,0.036259,0.122058,0.081272,0.126831,0.086470
243,20211028,300955,0.022382,0.079001,0.235551,0.087027,0.086132,0.180282,0.029612,0.029113,0.101804,0.062017,0.076216,0.046068
244,20211027,302549,0.028539,0.058406,0.208973,0.025151,0.080134,0.172465,0.003819,0.020679,0.065730,0.044808,0.078712,0.024755
245,20211026,304908,0.000000,0.063210,0.153057,0.000000,0.045180,0.129693,0.002620,0.018776,0.114359,0.017663,0.084169,0.000000


In [89]:
df_ind_1year_sc.to_csv('industry_top5stock_KOSPI_index.csv', index=False)